In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# set to use the live APIs
# os.environ["TEST_BIGQUERY_APIS_LIVE"] = "True"

In [ ]:
# isort: off
# stdlib
import os
import sys

# syft absolute
import syft as sy
from syft.util.util import (
    find_base_dir_with_tox_ini,
    get_caller_file_path,
    is_interpreter_jupyter,
)


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()
from syft import test_settings

# third party
from apis import make_schema
from apis import make_submit_query
from apis import make_test_query

# run email server
from email_helpers import get_email_server
# isort: on

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

# Launch server & login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
email_server, smtp_server = get_email_server()

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
this_worker_pool_name = "bigquery-pool"

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

# Create `test_query` endpoint

In [ ]:
# !pip list | grep bigquery

In [ ]:
# !uv pip install db-dtypes google-cloud-bigquery

In [ ]:
# Look up the worker pools and identify the name of the one that has the required packages
# After, bind the endpoint to that workerpool
high_client.worker_pools

In [ ]:
mock_func = make_test_query(
    settings={
        "rate_limiter_enabled": True,
        "calls_per_min": 10,
    }
)

In [ ]:
private_func = make_test_query(
    settings={
        "rate_limiter_enabled": False,
    }
)

In [ ]:
new_endpoint = sy.TwinAPIEndpoint(
    path="bigquery.test_query",
    description="This endpoint allows to query Bigquery storage via SQL queries.",
    private_function=private_func,
    mock_function=mock_func,
    worker_pool=this_worker_pool_name,
)

high_client.custom_api.add(endpoint=new_endpoint)

## Update `test_query` endpoint

In [ ]:
# Here, we update the endpoint to timeout after 100s (rather the default of 60s)
high_client.api.services.api.update(
    endpoint_path="bigquery.test_query", endpoint_timeout=120
)

In [ ]:
high_client.api.services.api.update(
    endpoint_path="bigquery.test_query", hide_mock_definition=True
)

## Test `test_query` endpoint

In [ ]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

In [ ]:
# Test mock version
result = high_client.api.services.bigquery.test_query.mock(
    sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
)
result

In [ ]:
assert len(result) == 10

In [ ]:
# Test private version
result = high_client.api.services.bigquery.test_query.private(
    sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 10"
)
result

In [ ]:
assert len(result) == 10

In [ ]:
# Test mock version for wrong queries
with sy.raises(
    sy.SyftException(public_message="*must be qualified with a dataset*"), show=True
):
    _ = high_client.api.services.bigquery.test_query.mock(
        sql_query="SELECT * FROM invalid_table LIMIT 1"
    )

In [ ]:
# Test private version
result = high_client.api.services.bigquery.test_query.private(
    sql_query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 1"
)
result

In [ ]:
assert len(result) == 1

## Inspect endpoint state

In [ ]:
# Inspect the context state on an endpoint
state = high_client.api.services.bigquery.test_query.mock.context.state
state

In [ ]:
len(state[ADMIN_EMAIL])

In [ ]:
assert len(state[ADMIN_EMAIL]) >= 2

# Create `schema` endpoint

In [ ]:
schema_function = make_schema(
    settings={
        "calls_per_min": 5,
    },
    worker_pool=this_worker_pool_name,
)

In [ ]:
high_client.custom_api.add(endpoint=schema_function)
high_client.refresh()

## Test `schema` endpoint

In [ ]:
# Testing schema
result = high_client.api.services.bigquery.schema()
result

In [ ]:
assert len(result) == 23

# Create `submit_query` endpoint

In [ ]:
submit_query_function = make_submit_query(
    settings={}, worker_pool=this_worker_pool_name
)

In [ ]:
high_client.custom_api.add(endpoint=submit_query_function)

In [ ]:
high_client.api.services.api.update(
    endpoint_path="bigquery.submit_query", hide_mock_definition=True
)

In [ ]:
high_client.custom_api.api_endpoints()

In [ ]:
assert len(high_client.custom_api.api_endpoints()) == 3

In [ ]:
high_client.api.services.bigquery.test_query

In [ ]:
high_client.api.services.bigquery.submit_query

## Test `submit_query` endpoint

In [ ]:
# Testing submit query
result = high_client.api.services.bigquery.submit_query(
    func_name="my_func",
    query=f"SELECT * FROM {dataset_1}.{table_1} LIMIT 1",
)

# Test emails

In [ ]:
# assert (
#     "Job Failed"
#     in email_server.get_emails_for_user(user_email=ADMIN_EMAIL)[0].email_content
# )

In [ ]:
# email_server.get_emails_for_user(user_email="admin@bigquery.org")

In [ ]:
# TODO: change this to be all admins or configure which ones etc

In [ ]:
# assert len(email_server.get_emails_for_user(user_email="admin@bigquery.org")) > 0

In [ ]:
# result

In [ ]:
# assert "Query submitted" in result

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()